In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import os
from v4vapp_backend_v2.config.setup import InternalConfig
from v4vapp_backend_v2.database.db import get_mongodb_client_defaults
from v4vapp_backend_v2.actions.tracked_models import TrackedBaseModel

target_dir = "/Users/bol/Documents/dev/v4vapp/v4vapp-backend-v2/"
os.chdir(target_dir)
print("Current working directory:", os.getcwd())


In [ ]:
from v4vapp_backend_v2.accounting.balance_sheet import (
    generate_balance_sheet_pandas,
    get_account_balance_printout,
    get_ledger_dataframe,
    list_all_accounts,
)


CONFIG = InternalConfig(config_filename="devhive.config.yaml").config

db_client = get_mongodb_client_defaults()

TrackedBaseModel.db_client = db_client
ledger_df = await get_ledger_dataframe()
balance_sheet_dict = await generate_balance_sheet_pandas(df=ledger_df)


In [ ]:
all_accounts = await list_all_accounts()
print("All accounts:", all_accounts)
balances = {}
for account in all_accounts:
    print(f"Account: {account}")
    balances[str(account)] = await get_account_balance_printout(
        account=account, full_history=True, df=ledger_df
    )
for account, balance in balances.items():
    print(f"Balance for {account}:")
    print(balance[0])


In [ ]:
from v4vapp_backend_v2.actions.hive_to_lightning import get_verified_hive_client
from nectar.account import Account
from nectar.amount import Amount
from v4vapp_backend_v2.accounting.account_type import LedgerAccount
from pprint import pprint

from v4vapp_backend_v2.helpers.crypto_prices import Currency
from math import isclose

print("Checking server and treasury accounts vs Hive balances...")
hive_client, server_name = await get_verified_hive_client()
server_account = Account(server_name, blockchain_instance=hive_client)
treasury_account = Account("devtre.v4vapp", blockchain_instance=hive_client)

# Server account balances
server_ledger_account = LedgerAccount.from_string(
    "Customer Deposits Hive (Asset) - Sub: devser.v4vapp"
)
server_summary_text, server_summary = await get_account_balance_printout(
    account=server_ledger_account,
    full_history=True,
)
print("Server Account Balance Summary:")

pprint("Server Account " + str(server_account.balances.get("available", [])))
if Currency.HIVE.value in server_summary.unit_summaries:
    print("HIVE Balance:", server_account.balances.get("available", Amount(0, "HIVE"))[0])
    pprint(server_summary.unit_summaries[Currency.HIVE.value].final_balance)
    if isclose(
        server_summary.unit_summaries[Currency.HIVE.value].final_balance,
        server_account.balances.get("available", Amount(0, "HIVE"))[0],
        abs_tol=0.002,  # absolute tolerance for 3 decimal places
    ):
        print("Balance match for HIVE Server Account")
    else:
        print("Balance mismatch for HIVE Server Account")
if Currency.HBD.value in server_summary.unit_summaries:
    print("HBD Balance:", server_account.balances.get("available", Amount(0, "HBD"))[1])
    pprint(server_summary.unit_summaries[Currency.HBD.value].final_balance)
    if isclose(
        server_summary.unit_summaries[Currency.HBD.value].final_balance,
        server_account.balances.get("available", Amount(0, "HBD"))[1],
        abs_tol=0.002,  # absolute tolerance for 3 decimal places
    ):
        print("Balance match for HBD Server Account")
    else:
        print("Balance mismatch for HBD Server Account")


treasury_ledger_account = LedgerAccount.from_string("Treasury Hive (Asset) - Sub: devtre.v4vapp")
pprint("Treasury Account " + str(treasury_account.balances.get("available", [])))
treasury_summary_text, treasury_summary = await get_account_balance_printout(
    account=treasury_ledger_account,
    full_history=True,
)
print("Treasury Account Balance Summary:")
if Currency.HIVE.value in treasury_summary.unit_summaries:
    print("HIVE Balance:", treasury_account.balances.get("available", Amount(0, "HIVE"))[0])
    pprint(treasury_summary.unit_summaries[Currency.HIVE.value].final_balance)
    if isclose(
        treasury_summary.unit_summaries[Currency.HIVE.value].final_balance,
        treasury_account.balances.get("available", Amount(0, "HIVE"))[0],
        abs_tol=0.002,  # absolute tolerance for 3 decimal places
    ):
        print("Balance match for HIVE Treasury Account")
    else:
        print("Balance mismatch for HIVE Treasury Account")
if Currency.HBD.value in treasury_summary.unit_summaries:
    print("HBD Balance:", treasury_account.balances.get("available", Amount(0, "HBD"))[1])
    pprint(treasury_summary.unit_summaries[Currency.HBD.value].final_balance)
    if isclose(
        treasury_summary.unit_summaries[Currency.HBD.value].final_balance,
        treasury_account.balances.get("available", Amount(0, "HBD"))[1],
        abs_tol=0.002,  # absolute tolerance for 3 decimal places
    ):
        print("Balance match for HBD Treasury Account")
    else:
        print("Balance mismatch for HBD Treasury Account")

In [ ]:
print(balance_sheet_dict)

In [ ]:
print(ledger_df)


In [ ]:
from datetime import datetime, timezone
from v4vapp_backend_v2.accounting.balance_sheet import get_ledger_entries
from v4vapp_backend_v2.accounting.balance_sheet import (
    balance_sheet_all_currencies_printout,
    balance_sheet_printout,
)

"""
All Balances sheets and all_accounts
"""
balance_sheet_currencies_str = balance_sheet_all_currencies_printout(balance_sheet_dict)
balance_sheet_str = balance_sheet_printout(balance_sheet_dict)


print("Ledger entries:")
ledger_entries = await get_ledger_entries(
    collection_name="ledger",
    filter_by_account=None,
    as_of_date=datetime.now(tz=timezone.utc),
)
for ledger_entry in ledger_entries:
    print(ledger_entry)

print("Balance sheet (currencies):")
print(balance_sheet_currencies_str)
print("Balance sheet:")
print(balance_sheet_str)

for account, (balance, summary) in balances.items():
    print(f"Balance for {account}:")
    print(balance)

In [ ]:
print(balances["Customer Liability Hive (Liability) - Sub: v4vapp-test"][0])


In [ ]:
print(balances["Customer Deposits Hive (Asset) - Sub: devser.v4vapp"][0])

In [ ]:
print(balances["Fee Income Lightning (Revenue) - Sub: umbrel"][0])

In [ ]:
print(balances["Treasury Lightning (Asset) - Sub: umbrel"])


In [ ]:
account = LedgerAccount.from_string("Converted Hive Offset (Asset) - Sub: devser.v4vapp")

(printout_str, data) = await get_account_balance_printout(account=account, full_history=True)
print(printout_str)

In [ ]:
ledger_df = await get_ledger_dataframe()
print("Ledger DataFrame:")
print(ledger_df.head())